# 1. Bộ dữ liệu

> * Bộ dữ liệu được sử dụng trong dự án này là ***một phần*** của một bộ dữ liệu gốc chứa các đánh giá sản phẩm và metadata từ Amazon, bao gồm **142.8 triệu** đánh giá từ tháng 5 năm 1996 đến tháng 7 năm 2014. [Amazon product data](https://cseweb.ucsd.edu/~jmcauley/datasets/amazon/links.html)
>   
> * Bộ dữ liệu này bao gồm các đánh giá (điểm đánh giá, văn bản, số phiếu bầu hữu ích), metadata sản phẩm (mô tả, thông tin danh mục, giá cả, thương hiệu, và đặc điểm hình ảnh), và liên kết (đồ thị sản phẩm được xem và được mua cùng).
>
> * Chúng ta làm việc với dữ liệu về các sản phẩm làm đẹp (Beauty Products), với 2 dataset như sau:



**1. meta_Beauty.json.gz**: chứa 259,204 thông tin về các sản phẩm.

**2. ratings_Beauty.csv**: chứa 2,023,070 đánh giá từ người dùng cho các sản phẩm.

### Mục tiêu: xây dựng Hệ gợi ý sản phẩm cho người dùng theo 2 phương pháp:
* Collaborative Filtering

* Content-based Filtering

# 2. Tiền xử lý dữ liệu

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("ggplot")

import sklearn
from sklearn.decomposition import TruncatedSVD

## 2.1. Đọc dữ liệu thô

In [38]:
amazon_ratings = pd.read_csv('ratings_Beauty.csv')
amazon_ratings = amazon_ratings.dropna()
amazon_ratings.head()


,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200


In [39]:
amazon_ratings.shape

(2023070, 4)

## 2.2. Tạo Utility Matrix

In [43]:
amazon_ratings1 = amazon_ratings.head(10000)

In [41]:
ratings_utility_matrix = amazon_ratings1.pivot_table(values='Rating', index='ProductId', columns='UserId', fill_value=0)
ratings_utility_matrix.head()

UserId,A00205921JHJK5X9LNP42,A024581134CV80ZBLIZTZ,A03056581JJIOL5FSKJY7,A03099101ZRK4K607JVHH,A0505229A7NSH3FRXRR4,A05492663T95KW63BR75K,A059547920Q3LZVFHLPI3,A07410232KYRFR25CIUGJ,A082796624UNM47DSAI6K,A0864963DOAY7LXGS5I6,...,AZW1HXXYAC15B,AZWRTJPN7NXT,AZWTXHXZXFAYP,AZYQEFB9Y5N22,AZZHB6U54UDYW,AZZHJZP4GQPPZ,AZZNK89PXD006,AZZOFVMQC0BJG,AZZQXL8VDCFTV,AZZTJQ7CQZUD8
ProductId,,,,,,,,,,,,,,,,,,,,,
0205616461,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0558925278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0733001998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0737104473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0762451459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
ratings_utility_matrix.shape

(886, 9697)

### Dùng TruncatedSVD để giảm chiều của Utility Matrix, giữ lại 10 thành phần (n_components=10) quan trọng nhất.

In [22]:
SVD = TruncatedSVD(n_components=10, random_state=42)
decomposed_matrix = SVD.fit_transform(ratings_utility_matrix)
decomposed_matrix.shape

(886, 10)

### Tính toán ma trận tương quan (Correlation Matrix)

In [23]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(886, 886)

# 3. Collaborative Filtering Model

## 3.1.1 Item-item Collaborative Filtering.

### Tìm chỉ số của sản phẩm có ProductId = i

In [72]:
ratings_utility_matrix.index[10]

'130414674X'

In [97]:
i = "130414674X"

product_names = list(ratings_utility_matrix.index)
product_ID = product_names.index(ratings_utility_matrix.index[10])

product_ID

10

### Tính toán độ tương quan của các sản phẩm còn lại với sản phẩm ProductId = i

In [98]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

(886,)

### Gợi ý 10 sản phẩm có độ tương quan cao nhất và lớn hơn 0.95

In [99]:
Recommend = list(ratings_utility_matrix.index[correlation_product_ID > 0.95])

# Removes the item already bought by the customer
Recommend.remove(i) 

Recommend[0:9]

['0205616461',
 '0558925278',
 '1304139220',
 '130414089X',
 '1304174778',
 '1304196046',
 '1304196062',
 '1304196135',
 '1304482634']

## 3.1.2. Đánh giá mô hình.

In [101]:
# Chú ý: Mã này là mã giả, vì ta không có thông tin chi tiết về sở thích cá nhân cụ thể của từng người dùng
important_products = amazon_ratings[amazon_ratings['Rating'] >= 4].groupby('UserId')['ProductId'].apply(set).to_dict()


In [103]:
# Mã giả, cần thông tin cụ thể về người dùng mục tiêu
user_target = 'A39HTATAQ9V7YF'

# Lấy sản phẩm quan trọng cho người dùng mục tiêu
true_labels = important_products.get(user_target, set())

# Tính precision và recall
true_positives = set(Recommend) & true_labels
precision = len(true_positives) / len(Recommend) if Recommend else 0
recall = len(true_positives) / len(true_labels) if true_labels else 0

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')


Precision: 0.0043
Recall: 0.3333


# Khó khăn cần tiếp tục giải quyết
### Precision và Recall thấp, cần tìm hiểu nguyên nhân

## 3.2. User-User Collaborative Filtering. (*incomplete*)

# 4. Collaborative Filtering Model (*incomplete*)

In [2]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('meta_Beauty.json.gz')



In [3]:
df

,asin,description,title,imUrl,salesRank,categories,price,related,brand
0,0205616461,"As we age, our once youthful, healthy skin suc...",Bio-Active Anti-Aging Serum (Firming Ultra-Hyd...,http://ecx.images-amazon.com/images/I/41DecrGO...,{'Health & Personal Care': 461765},"[[Beauty, Skin Care, Face, Creams & Moisturize...",NaN,NaN,NaN
1,0558925278,Mineral Powder Brush--Apply powder or mineral ...,Eco Friendly Ecotools Quality Natural Bamboo C...,http://ecx.images-amazon.com/images/I/51L%2BzY...,{'Beauty': 402875},"[[Beauty, Tools & Accessories, Makeup Brushes ...",NaN,NaN,NaN
2,0733001998,"From the Greek island of Chios, this Mastiha b...",Mastiha Body Lotion,http://ecx.images-amazon.com/images/I/311WK5y1...,{'Beauty': 540255},"[[Beauty, Skin Care, Body, Moisturizers, Lotio...",NaN,NaN,NaN
3,0737104473,Limited edition Hello Kitty Lipstick featuring...,Hello Kitty Lustre Lipstick (See sellers comme...,http://ecx.images-amazon.com/images/I/31u6Hrzk...,{'Beauty': 931125},"[[Beauty, Makeup, Lips, Lipstick]]",NaN,NaN,NaN
4,0762451459,"The mermaid is an elusive (okay, mythical) cre...",Stephanie Johnson Mermaid Round Snap Mirror,http://ecx.images-amazon.com/images/I/41y2%2BF...,NaN,"[[Beauty, Tools & Accessories, Mirrors, Makeup...",19.98,NaN,NaN
...,...,...,...,...,...,...,...,...,...
259199,B00LP2YB8E,Color: White\nFullness72 inches\nCenter Gather...,2t 2t Edge Crystal Rhinestones Bridal Wedding ...,http://ecx.images-amazon.com/images/I/41E630m-...,NaN,"[[Beauty, Hair Care, Styling Tools, Styling Ac...",NaN,NaN,NaN
259200,B00LOS7MEE,"The secret to long lasting colors, healthy nai...",French Manicure Gel Nail Polish Set - &quot;Se...,http://ecx.images-amazon.com/images/I/41skHL1O...,{'Beauty': 108820},"[[Beauty, Makeup, Nails, Nail Polish]]",NaN,"{'also_viewed': ['B0057JCYYE', 'B00LMXHR1Y', '...",NaN
259201,B00LPVG6V0,ResQ Organics Face & Body Wash - With Aloe Ver...,ResQ Organics Face &amp; Body Wash - Aloe Vera...,http://ecx.images-amazon.com/images/I/31C1w4Ku...,NaN,"[[Beauty, Skin Care, Face, Creams & Moisturize...",NaN,NaN,NaN
259202,B00LTDUHJQ,Color: White\n2 Tier \nFullness 72 inches\nSew...,2 Tier Tulle Elbow Wedding Veil with Ribbon Ed...,http://ecx.images-amazon.com/images/I/51%2B%2B...,NaN,"[[Beauty, Hair Care, Styling Tools, Styling Ac...",NaN,NaN,NaN


In [4]:
import pandas as pd

# Giả sử 'df' là DataFrame của bạn

# Bước 1 & 2: Gộp và phẳng hóa list của các categories
all_categories = [category for sublist in df['categories'] for category in sublist[0]]

# Bước 3: Loại bỏ các giá trị trùng lặp và đếm
unique_categories = set(all_categories)  # Sử dụng set để loại bỏ trùng lặp
num_unique_categories = len(unique_categories)  # Đếm số lượng phần tử duy nhất

print(f"Số lượng categories: {num_unique_categories}")
print("Danh sách các categories:")
for category in unique_categories:
    print(category)


Số lượng categories: 275
Danh sách các categories:
Nails
Bronzing Powder
Masks
Maternity
Shampoo Plus Conditioner
Bath Bombs
Home Permanent Kits
Microdermabrasion
Decorative Combs
Bathing Accessories
Self Tanners
Combinations
Sunscreens
Household Cleaning
Body Paint
Tools & Accessories
Loofahs, Sponges & Poufs
Cuticle Repair
Nail Art Equipment
Concealer & Base
Styling Tools
Tote Bags
Shampoo & Conditioner Sets
Chemical Hair Dyes
Cuticle Pushers
Bath Pillows
Lip Glosses
Day Creams
Hot & Cold Therapies
Men's Deodorants & Antiperspirants
Sports
Glitter & Shimmer
Body Mud
Liner & Shadow Combinations
Curlers
Lactic Acid
Cuticle Care
Hair Relaxers
Hand Creams & Lotions
Washes
Cuticle Scissors
Decorations
Nail Dryers
Hair Extensions & Wigs
Curl Enhancers
Dark Circle Treatments
Diffusers
Moisturizing Gloves
Toe Separators
Makeup Mirrors
Scrubs & Body Treatments
Hair Dryers
Candles & Home Scents
Claws
Puffiness Treatments
Combs
Night Creams
Lotions
Conditioners
Hair Pins
Clippers & Trimmers
Hou

In [5]:
# Làm sạch dữ liệu văn bản và xử lý giá trị thiếu
df['description'] = df['description'].fillna('').map(lambda x: x.lower())
df['title'] = df['title'].fillna('').map(lambda x: x.lower())
df['price'] = df['price'].fillna(df['price'].mean())

# Đối với `categories`, giả sử mỗi item chỉ có một danh sách category duy nhất
df['categories'] = df['categories'].map(lambda x: ','.join(x[0]).lower())


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Khởi tạo TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # Giới hạn số lượng features

# Kết hợp văn bản từ description và title
text_features = tfidf_vectorizer.fit_transform(df['description'] + " " + df['title'])

# Có thể chuyển đổi `text_features` thành array để dễ dàng kết hợp với các feature khác nếu cần


In [7]:
from sklearn.preprocessing import OneHotEncoder

# Khởi tạo One-Hot Encoder
onehot_encoder = OneHotEncoder(handle_unknown='ignore')

# Encode `categories`
categories_features = onehot_encoder.fit_transform(df[['categories']])


In [20]:
import numpy as np

# Chuyển đổi `price` thành array để kết hợp với các feature khác
price_features = np.array(df['price']).reshape(-1, 1)


In [22]:
from scipy.sparse import hstack

# Kết hợp các feature
final_features = hstack([text_features, categories_features, price_features])


In [23]:
final_features_dense = final_features.toarray()

# Xem kích thước của final_features
print("Kích thước của final_features:", final_features_dense.shape)



Kích thước của final_features: (259204, 1285)


In [24]:
# Xem một số giá trị mẫu từ final_features
print("Một số giá trị mẫu từ final_features:\n", final_features_dense[:1000])

Một số giá trị mẫu từ final_features:
 [[ 0.          0.          0.         ...  0.          0.
  24.87816496]
 [ 0.          0.          0.         ...  0.          0.
  24.87816496]
 [ 0.          0.          0.         ...  0.          0.
  24.87816496]
 ...
 [ 0.          0.          0.         ...  0.          0.
   3.07      ]
 [ 0.          0.          0.         ...  0.          0.
  24.87816496]
 [ 0.          0.          0.         ...  0.          0.
   9.74      ]]
